## OpenSearch - Vector Store
OpenSearch는 대규모 데이터셋에 대한 유사도 검색을 위한 강력한 엔진입니다. Amazon OpenSearch Service를 통해 쉽게 클라우드 환경에서도 이용할 수 있습니다. 이와 함께 Vector Store를 사용하면 고차원 벡터 데이터를 효율적으로 저장하고 빠르게 검색할 수 있어, 복잡한 자연어 처리 작업을 더욱 간편하게 수행할 수 있습니다.

* Container: `Data Science 3.0` (studio, python 3.10), `conda_python3` (notebook)

In [3]:
%store -r endpoint_name_emb endpoint_name_text
try:
    endpoint_name_emb
    endpoint_name_text
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] TASK-1, TASK-2 노트북을 다시 실행해 주세요.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

# 1. SageMaker Endpoint Wrapper

### 1.1. SageMaker LLM Endpoint Wrapper

In [4]:
import sys
%load_ext autoreload
%autoreload 2
sys.path.append('./utils') # src 폴더 경로 설정
import json
import time
import boto3
import botocore
import numpy as np
from inference_utils import Prompter
from typing import Any, Dict, List, Optional
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.llms.sagemaker_endpoint import LLMContentHandler, SagemakerEndpoint
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
prompter = Prompter("kullm")
params = {
      'do_sample': False,
      'max_new_tokens': 128,
      'temperature': 1.0,
      'top_k': 0,
      'top_p': 0.9,
      'return_full_text': False,
      'repetition_penalty': 1.1,
      'presence_penalty': None,
      'eos_token_id': 2
}

class KullmContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        '''
        입력 데이터 전처리 후에 리턴
        '''
        context, question = prompt.split("||SPEPERATOR||") 
        prompt = prompter.generate_prompt(question, context)

        print ("prompt", prompt)
        payload = {
            'inputs': [prompt],
            'parameters': model_kwargs
        }
                           
        input_str = json.dumps(payload)
        
        return input_str.encode('utf-8')
    

    def transform_output(self, output: bytes) -> str:
        
        response_json = json.loads(output.read().decode("utf-8"))              
        generated_text = response_json[0][0]["generated_text"]
        
        return generated_text    

In [6]:
aws_region = boto3.Session().region_name
LLMTextContentHandler = KullmContentHandler()
seperator = "||SPEPERATOR||"

llm_text = SagemakerEndpoint(
    endpoint_name=endpoint_name_text,
    region_name=aws_region,
    model_kwargs=params,    
    content_handler=LLMTextContentHandler,
)

### 1.2. SageMaker Embedding Model Endpoint Wrapper

In [7]:
class SagemakerEndpointEmbeddingsJumpStart(SagemakerEndpointEmbeddings):
    def embed_documents(self, texts: List[str], chunk_size: int=1) -> List[List[float]]:
        """Compute doc embeddings using a SageMaker Inference Endpoint.

        Args:
            texts: The list of texts to embed.
            chunk_size: The chunk size defines how many input texts will
                be grouped together as request. If None, will use the
                chunk size specified by the class.

        Returns:
            List of embeddings, one for each text.
        """
        results = []
        _chunk_size = len(texts) if chunk_size > len(texts) else chunk_size
        
        print("text size: ", len(texts))
        print("_chunk_size: ", _chunk_size)

        for i in range(0, len(texts), _chunk_size):
            
            #print (i, texts[i : i + _chunk_size])
            response = self._embedding_func(texts[i : i + _chunk_size])
            #print (i, response, len(response[0].shape))
            
            results.extend(response)
        return results

In [8]:
class KoSimCSERobertaContentHandler(EmbeddingsContentHandler):
    
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        
        input_str = json.dumps({"inputs": prompt, **model_kwargs})
        
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        
        response_json = json.loads(output.read().decode("utf-8"))
        ndim = np.array(response_json).ndim    
        
        if ndim == 4:
            # Original shape (1, 1, n, 768)
            emb = response_json[0][0][0]
            emb = np.expand_dims(emb, axis=0).tolist()
        elif ndim == 2:
            # Original shape (n, 1)
            emb = []
            for ele in response_json:
                e = ele[0][0]
                emb.append(e)
        else:
            print(f"Other # of dimension: {ndim}")
            emb = None
        return emb

In [ ]:
LLMEmbHandler = KoSimCSERobertaContentHandler()

In [9]:
LLMEmbHandler = KoSimCSERobertaContentHandler()

llm_emb = SagemakerEndpointEmbeddingsJumpStart(
    endpoint_name=endpoint_name_emb,
    region_name=aws_region,
    content_handler=LLMEmbHandler,
)

# 2 Create OpenSearch domain
* Follow below
    - https://docs.aws.amazon.com/ko_kr/opensearch-service/latest/developerguide/gsgcreate-domain.html
* Add policy (using SDK)
    - AmazonOpenSearchServiceFullAccess

**step 1. opensearch console로 이동 후 Navigator에서 Domain 이동 후 Create domain 선택** <BR>

<div align="center">
    <img src="./images/open1.png" alt="Step 1">
</div>
    
**step 2. domain config 셋팅** <BR>
    
* Domain name : 
* Domain creation Method: 사용자 지정생성 (손쉬운생성 선택시 '최대 절수' 오류 발생하는 경우)
<div align="center">
    <img src="./images/open2.png" alt="Step 2">
</div>
    

* Engine options: OpenSearch_2.7
* Network: Public access
<div align="center">
    <img src="./images/open3.png" alt="Step 4">
</div>
* Master user: Create master user
* Master username, Master password and Confirm master password 입력
<div align="center">
    <img src="./images/open4.png" alt="Step 4">
</div>
* 고급클러스터 > 최대절수 선택(손쉬운생성 오류경우)
<div align="center">
    <img src="./images/open5.png" alt="Step 5">
</div>    
* 오른쪽 아래 주황색 create 선택



**step 3. access설정** <BR>

* 도메인  보안구성 > 편집 클릭

<div align="center">
    <img src="./images/open6.png" alt="Step 6">
</div>  

* 도메인 수준 엑세스 정책 구성 > Effect : Allow 로 수정 

<div align="center">
    <img src="./images/open7.png" alt="Step 7">
</div>  

**step 4.Domain enapoint 복사** <BR>

<div align="center">
    <img src="./images/open8.png" alt="Step 8">
</div>  

* create_domain: https://boto3.amazonaws.com/v1/documentation/api/1.18.51/reference/services/opensearch.html#OpenSearchService.Client.create_domain
*     
**It takes about 20 mins**

In [16]:
opensearch_domain_endpoint = "https://search-test-wzntp6rqfgbn4s4zr2zvt5nhdq.us-east-1.es.amazonaws.com"

In [26]:
http_auth = ("raguser", "MarsEarth1!") # Master username, Master password

### 2.2. load context files and build indexer
We are now ready to create scripts which will read data from the local directory, use langchain to create embeddings and then upload the embeddings into OpenSearch.

In [18]:
import json
import boto3
from langchain.document_loaders.csv_loader import CSVLoader

In [19]:
loader = CSVLoader(
    file_path="./dataset/fsi_smart_faq_ko.csv",
    source_column="Source",
    encoding="utf-8"
)
context_documents = loader.load()

In [20]:
len(context_documents), context_documents[5]

(89,
 Document(page_content='no: 84\nCategory: 기존 공동인증서를 보유한 상태에서 금융인증서 발급이 가능한가요?\nInformation: 공동인증서와 금융인증서는 별개의 인증서로 두 가지 인증서를 모두 사용할 수 있습니다.\ntype: 금융인증서\nSource: 신한은행', metadata={'source': '신한은행', 'row': 5}))

## 2.3. OpenSearch에 Data 입력

이 스크립트는 모든 것을 하나로 모으고 문서를 청크로 나눈 다음 langchain 패키지를 사용하여 임베딩을 생성한 다음(`SagemakerEndpointEmbeddingsJumpStart`를 통해) `OpenSearchVectorSearch`를 사용하여 OpenSearch에 데이터를 수집합니다.

단순하게 유지하기 위해 청크 크기는 800개 토큰의 고정 길이로 설정되고 200개 토큰이 중복됩니다. langchain `OpenSearchVectorSearch`는 `opensearch-py` 패키지에 대한 래퍼를 제공합니다. 단일 PUT 요청에서 여러 레코드를 수집하기 위해 `/_bulk` API 엔드포인트를 사용합니다.

In [21]:
import time
import pprint
import logging
import sagemaker
from langchain.vectorstores import OpenSearchVectorSearch
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [22]:
#pp = pprint.PrettyPrinter(indent=4)

In [23]:
# global constants
logger = logging.getLogger()
logging.basicConfig(format='%(asctime)s,%(module)s,%(processName)s,%(levelname)s,%(message)s', level=logging.INFO, stream=sys.stderr)

role = sagemaker.get_execution_role()
role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


'arn:aws:iam::143656149352:role/service-role/AmazonSageMaker-ExecutionRole-20220317T150353'

### OpenSearch에 Index 생성 및 Vector Store 데이터 저장 전송

In [24]:
index_name = "fsi-sample"

In [27]:
%%time
logger.info('Loading documents ...')
docs = loader.load()

# # add a custom metadata field, such as timestamp
for doc in docs:
    doc.metadata['timestamp'] = time.time()
    doc.metadata['embeddings_model'] = endpoint_name_emb

text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

# by default langchain would create a k-NN index and the embeddings would be ingested as a k-NN vector type
docsearch = OpenSearchVectorSearch.from_documents(
    index_name=index_name,
    documents=documents,
    embedding=llm_emb,
    opensearch_url=opensearch_domain_endpoint,
    http_auth=http_auth,
    bulk_size=10000,
    timeout=60
)

2023-09-19 07:03:28,670,<timed exec>,MainProcess,INFO,Loading documents ...


text size:  90
_chunk_size:  1


2023-09-19 07:03:44,428,base,MainProcess,WARNING,GET https://search-rag-hol-user1-zrawnyzvdxkws34crzt42cg454.us-east-1.es.amazonaws.com:443/fsi-sample [status:404 request:0.795s]
2023-09-19 07:03:44,760,base,MainProcess,INFO,PUT https://search-rag-hol-user1-zrawnyzvdxkws34crzt42cg454.us-east-1.es.amazonaws.com:443/fsi-sample [status:200 request:0.331s]
2023-09-19 07:03:45,296,base,MainProcess,INFO,POST https://search-rag-hol-user1-zrawnyzvdxkws34crzt42cg454.us-east-1.es.amazonaws.com:443/_bulk [status:200 request:0.504s]
2023-09-19 07:03:45,416,base,MainProcess,INFO,POST https://search-rag-hol-user1-zrawnyzvdxkws34crzt42cg454.us-east-1.es.amazonaws.com:443/_bulk [status:200 request:0.108s]
2023-09-19 07:03:45,730,base,MainProcess,INFO,POST https://search-rag-hol-user1-zrawnyzvdxkws34crzt42cg454.us-east-1.es.amazonaws.com:443/fsi-sample/_refresh [status:200 request:0.313s]


CPU times: user 4.86 s, sys: 207 ms, total: 5.07 s
Wall time: 17.1 s


## 5. QnA

In [28]:
from functools import lru_cache
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

### 5.1. Query and Response

In [29]:
import copy
import functools
import concurrent.futures

In [30]:
prompt_template = ''.join(["{context}", seperator, "{question}"])
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain = load_qa_chain(llm=llm_text, chain_type="stuff", prompt=PROMPT, verbose=True)

In [31]:
vectro_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)

In [32]:
def pretty_print_documents(response):
    for doc, score in response:
        print(f'\nScore: {score}')
        print(f'Document Number: {doc.metadata["row"]}')
        print(f'Source: {doc.metadata["source"]}')

        # Split the page content into lines
        lines = doc.page_content.split("\n")

        # Extract and print each piece of information if it exists
        for line in lines:
            split_line = line.split(": ")
            if len(split_line) > 1:
                print(f'{split_line[0]}: {split_line[1]}')

        print('-' * 50)

In [33]:
def filter_and_remove_score_opensearch_vector_score(res, cutoff_score = 0.006, variance=0.95):
    # Get the lowest score
    highest_score = max(score for doc, score in res)
    print('highest_score : ', highest_score)
    # If the lowest score is over 200, return an empty list
    if highest_score < cutoff_score:
        return []
    # Calculate the upper bound for scores
    lower_bound = highest_score * variance
    print('lower_bound : ', lower_bound)
    # Filter the list and remove the score
    res = [doc for doc, score in res if score >= lower_bound]

    return res


def get_similiar_docs(query, k=5, fetch_k=300, score=True, bank=""):

    
    #query = f'{bank}, {query}'
    print (query)
    
    if score:
        pre_similar_doc = vectro_db.similarity_search_with_score(
            query,
            k=k,
            fetch_k=fetch_k,
            search_type="approximate_search", # approximate_search, script_scoring, painless_scripting
            space_type="l2",     #"l2", "l1", "linf", "cosinesimil", "innerproduct", "hammingbit";
            pre_filter={"bool": {"filter": {"term": {"text": bank}}}},
            boolean_filter={"bool": {"filter": {"term": {"text": bank}}}}
            #filter=dict(source=bank)
        )
        #print('jhs : ', similar_docs)
        pretty_print_documents( pre_similar_doc)
        similar_docs=filter_and_remove_score_opensearch_vector_score(pre_similar_doc)        
    else:
        similar_docs = vectro_db.similarity_search(
            query,
            k=k,
            search_type="approximate_search", # approximate_search, script_scoring, painless_scripting
            space_type="12",     #"l2", "l1", "linf", "cosinesimil", "innerproduct", "hammingbit";
            pre_filter={"bool": {"filter": {"term": {"text": bank}}}},
            boolean_filter={"bool": {"filter": {"term": {"text": bank}}}}
            
        )
    similar_docs_copy = copy.deepcopy(similar_docs)
    
    #print('similar_docs_copy : \n', similar_docs_copy)
    
    return similar_docs_copy


def get_answer(query, bank="",score=False, fetch_k=300, k=1):
                
    search_query = query
    
    similar_docs = get_similiar_docs(search_query, k=k,score=score, bank=bank)
    

    llm_query = '고객 서비스 센터 직원처럼, '+query+' 카테고리에 대한 Information을 찾아서 설명해주세요.'
    
    if not similar_docs:
        llm_query = query

    answer = chain.run(input_documents=similar_docs, question=llm_query)
    
    return answer

In [34]:
question ='안녕하세요. 날씨가 참 좋네요.'
response = get_answer(question, bank='신한은행',score=True, k=4)
print("챗봇 : ", response)

안녕하세요. 날씨가 참 좋네요.


2023-09-19 07:03:46,034,base,MainProcess,INFO,POST https://search-rag-hol-user1-zrawnyzvdxkws34crzt42cg454.us-east-1.es.amazonaws.com:443/fsi-sample/_search [status:200 request:0.124s]



Score: 0.003389907
Document Number: 19
Source: 신한은행
no: 70
Category: 홈페이지상에 제가 등록한 칭찬/불만/제안사항 조회할 수 있나요?
Information: 로그인 후 등록한 접수내용에 대해서 확인 가능합니다.
type: 홈페이지
Source: 신한은행
--------------------------------------------------

Score: 0.0032900402
Document Number: 82
Source: 신한은행
no: 7
Category: 인터넷으로 신규 예/적금 신청하는 방법을 알려주세요
Information: 인터넷상으로 예금/신탁을 신규가입하시려면 우선 고객님께서는인터넷뱅킹에 가입하셔야 하며 신규방법은 두 가지가 있습니다.1. 인터넷뱅킹에서 가입인터넷뱅킹 로그인을 하신 후 예금/신탁 > 신규 메뉴에서 예금 및 신탁 상품을 신규하실 수 있습니다.2. 신한S뱅크에서 가입신한S뱅크 상품센터 > 예금센터 메뉴에서 예금상품을 신규하실 수 있습니다.
type: 인터넷뱅킹
Source: 신한은행
--------------------------------------------------

Score: 0.0032718112
Document Number: 49
Source: 신한은행
no: 40
Category: 회원탈퇴 후 메일이 계속와요.
Information: 인터넷뱅킹가입을 하시면 예금/대출/카드 등 거래에 대한 안내(예:예금만기 등)외에 영업점안내메일 등 몇가지 부가서비스가 기본제공됩니다. 부가서비스는 홈페이지에 로그인하셔서(인터넷뱅킹사용자는 별도 회원가입이 필요없습니다.) 이메일서비스의 수신/거부 등 변경을 하시면 됩니다. 다만, 인터넷뱅킹을 해지하시더라도 예금,카드,대출 등의 거래가 남아있을 수 있기에 메일서비스는 계속 제공됩니다. 따라서 고객님의 경우에는 기존에 제공되는 메일서비스가 계속 남아있어 부가서비스 메일을 받으신 것이며. 정보가 유출되거나 하는 경우는 절대 없으니 안

In [35]:
q ='간편조회서비스는 회원가입해야하나요?'
response = get_answer(q, bank='신한은행',score=True, k=5)

print("챗봇 : ", response)

2023-09-19 07:03:46,814,base,MainProcess,INFO,POST https://search-rag-hol-user1-zrawnyzvdxkws34crzt42cg454.us-east-1.es.amazonaws.com:443/fsi-sample/_search [status:200 request:0.036s]


간편조회서비스는 회원가입해야하나요?

Score: 0.009032617
Document Number: 17
Source: 신한은행
no: 72
Category: 간편조회서비스는 회원가입해야만 이용할 수 있나요?
Information: 간편조회서비스에는 로그인을 위해 회원가입이 필요한 서비스와 회원가입 없이 누구나 이용 가능한 서비스가 있습니다.
type: 간편서비스
Source: 신한은행
--------------------------------------------------

Score: 0.007898479
Document Number: 29
Source: 신한은행
no: 60
Category: 홈페이지에서 계좌거래내역 조회할 수 있나요?
Information: 홈페이지회원  가입 후 간편조회서비스를 통해 계좌 거래내역 조회 가능합니다.
type: 간편서비스
Source: 신한은행
--------------------------------------------------

Score: 0.0072306204
Document Number: 16
Source: 신한은행
no: 73
Category: 아이핀으로 홈페이지회원 가입한 고객은 간편조회서비스 이용할 수 없나요?
Information: 아이핀으로 홈페이지 회원가입하신 고객은 홈페이지에서 개인회원으로 전환 후 간편조회서비스 이용가능 합니다. 
type: 간편서비스
Source: 신한은행
--------------------------------------------------

Score: 0.0070149396
Document Number: 46
Source: 신한은행
no: 43
Category: 홈페이지 회원가입은 어떻게 해야 하나요?
Information: 홈페이지 상단에 있는 [고객센터]를 클릭하여, 회원서비스 → 회원가입메뉴에서 회원가입 메뉴를 이용하시기 바랍니다. 기타 문의는 콜센터 1599-8000번으로 문의 바랍니다.
type: 홈페이지
Source: 신한은행
---------------

## 6. Cleanup

### 6.1. delete opensearch domain

In [38]:
client = boto3.client('opensearch')
response = client.delete_domain(
    DomainName=domain_name
)